In [1]:
from fontTools.ttLib import TTFont
from PIL import Image, ImageFont, ImageDraw
import numpy as np
from pathlib import Path
import os

In [2]:
def get_defined_chars(fontfile):
    # Get all defined characters from the font file
    ttf = TTFont(fontfile)
    chars = [chr(y) for y in ttf["cmap"].tables[0].cmap.keys()]
    return chars

In [3]:
def get_filtered_chars(fontpath):
    # Filter characters that can be rendered from the font
    ttf = read_font(fontpath)
    defined_chars = get_defined_chars(fontpath)
    avail_chars = []

    for char in defined_chars:
        img = np.array(render(ttf, char))
        if img.mean() != 255.:
            avail_chars.append(char.encode('utf-16', 'surrogatepass').decode('utf-16'))

    return avail_chars

In [4]:
def read_font(path, size=150):
    # Read the font with a specified size
    font = ImageFont.truetype(path, size)
    return font

In [5]:
def render(font, char, size=(128, 128), pad=20, bottom_pad=20, scale=0.55):
    # Scale the font size
    font_size = int(font.size * scale)
    font = ImageFont.truetype(font.path, font_size)

    # Create an image with enough space to center the text
    img = Image.new("L", (size[0] + 2*pad, size[1] + pad + bottom_pad), 255)
    draw = ImageDraw.Draw(img)

    # Get the bounding box for the text to be drawn
    bbox = draw.textbbox((0, 0), char, font=font)

    # Calculate the starting coordinates to center the text
    start_x = (img.width - bbox[2] - bbox[0]) / 2
    start_y = (img.height - bbox[3] - bbox[1]) / 2

    # Draw the text at the center of the image
    draw.text((start_x, start_y), char, font=font, fill=0)

    # Crop the image leaving the specified padding
    img = img.crop((pad, pad, size[0] + pad, size[1] + pad + bottom_pad - pad))
    return img

In [ ]:
# Character ranges for English, Russian, and Kazakh alphabets
english_chars = list(range(65, 91)) + list(range(97, 123))
russian_chars = list(range(0x0410, 0x0450)) + [0x0401, 0x0451]
kazakh_chars = [0x04D8, 0x04D9, 0x0492, 0x0493, 0x049A, 0x049B, 0x04A2, 0x04A3, 0x04E8, 0x04E9, 0x04B0, 0x04B1, 0x04AE, 0x04AF, 0x04BA, 0x04BB, 0x0406, 0x0456]

# Combined character list for generating images
my_gen_chars = [chr(i) for i in kazakh_chars + russian_chars + english_chars]

# Directory paths for saving generated images and font files
save_dir = Path(r"path/where/you/want/to/save")
save_dir.mkdir(parents=True, exist_ok=True)
font_dir = r"path/to/ttf/files"

# List all font files in the specified directory
custom_fonts = os.listdir(font_dir)

for font_file in custom_fonts:
    try:
        source_font = read_font(str(font_dir + '/' + font_file))
        gen_chars = np.intersect1d(get_filtered_chars(str(font_dir + '/' + font_file)), my_gen_chars)
        for char in gen_chars:
            try:
                font_char_dir = save_dir / font_file.split('.')[0]
                font_char_dir.mkdir(parents=True, exist_ok=True)
                source_img = render(source_font, char)
                prefix = 'upper_' if char.isupper() else 'lower_'
                path = font_char_dir / f"{prefix}{char}_{font_file.split('.')[0]}.png"
                source_img.save(path)
            except OSError as e:
                print(f"Skipping character {char} in font {font_file} due to error: {e}")
    except Exception as e:
        print(f"Failed to read or process font {font_file} due to error: {e}")